# open-source-marginal-emissions

## weather_data_retrieval

The purpose of this notebook is to provide an interactive way to retrieve data from either:
* CDS API (Copernicus Data Store) for ERA5 data
* Open-Meteo API

## Code

### Libraries

In [ ]:
from __future__ import annotations

import os
import sys
import time
import json
import math
import hashlib
import getpass
import calendar
import requests
from dataclasses import dataclass
from datetime import datetime, timedelta
from pathlib import Path
from typing import List, Tuple, Optional, Iterable

import cdsapi
from tqdm import tqdm

# Optional MPI
try:
    from mpi4py import MPI  # type: ignore
    MPI_AVAILABLE = True
except Exception:
    MPI_AVAILABLE = False

from concurrent.futures import ThreadPoolExecutor, as_completed


### Paths

In [2]:
root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
data_dir = os.path.join(root_dir, "data")
raw_data_dir = os.path.join(data_dir, "raw")

print(raw_data_dir)

/Users/Daniel/Desktop/open-source-marginal-emissions/data/raw


### Implementation

#### Inputs

##### User Defined

In [ ]:
invalid_era5_world_variables = [
    'fraction_of_cloud_cover',
    'large_scale_precipitation'
]

# - TO BE IMPLEMENTED -
invalid_era5_land_variables = [
]
invalid_open_meteo_variables = [
]

##### Accepted Values & Mappings

In [ ]:
NORMALIZATION_MAP = {
    "data_provider": {
        "cds": "cds",
        "copernicus": "cds",
        "copernicus climate": "cds",
        "copernicus data store": "cds",
        "copernicus climate data store": "cds",
        "era5": "cds",
        "1": "cds",
        "open-meteo": "open-meteo",
        "openmeteo": "open-meteo",
        "om": "open-meteo",
        "open": "open-meteo",
        "2": "open-meteo",

    },
    "era5_dataset_short_name": {
        "era5land": "era5-land",
        "era5-land": "era5-land",
        "land": "era5-land",
        "0.1": "era5-land",     # resolution of the dataset in ESPG:4326
        "1": "era5-land",
        "era5-world": "era5-world",
        "era5_world": "era5-world",
        "world": "era5-world",
        "era5": "era5-world",
        "0.25": "era5-world",   # resolution of the dataset in ESPG:4326
        "2" : "era5-world",
    },
    "open_meteo_dataset_short_name": {
        # NOT YET IMPLEMENTED
    },
    "boolean": {
        "yes": True, "y": True, "1": True, "true": True, "t": True, "on": True,
        "no": False, "n": False, "0": False, "false": False, "f": False, "off": False
    },
    "confirmation": {  # for yes/no questions
        "y": "yes", "yes": "yes", "ok": "yes", "sure": "yes", "confirm": "yes",
        "n": "no", "no": "no", "nah": "no", "never": "no"
    }

}


In [ ]:
class SessionState:
    def __init__(self):
        # Order matters – matches prompt flow
        self.fields = {
            "data_provider": {"value": None, "filled": False},
            "dataset_short_name": {"value": None, "filled": False},
            "api_url": {"value": None, "filled": False},
            "api_key": {"value": None, "filled": False},
            "config_source": {"value": None, "filled": False},
            "start_date": {"value": None, "filled": False},
            "end_date": {"value": None, "filled": False},
            "region_bounds": {"value": None, "filled": False},
            "variables": {"value": None, "filled": False},
            "save_dir": {"value": None, "filled": False},
            "parallel_settings": {"value": None, "filled": False},
            "retry_settings": {"value": None, "filled": False},  # NEW
        }

    def set(self, key, value):
        if key in self.fields:
            self.fields[key]["value"] = value
            self.fields[key]["filled"] = True

    def unset(self, key):
        if key in self.fields:
            self.fields[key]["value"] = None
            self.fields[key]["filled"] = False

    def get(self, key):
        return self.fields[key]["value"] if key in self.fields else None

    def previous_key(self, current_key):
        keys = list(self.fields.keys())
        idx = keys.index(current_key)
        return keys[idx - 1] if idx > 0 else None

    def first_unfilled_key(self):
        """
        Return the first key in the ordered fields that is not filled.
        This enables a simple wizard-like progression and supports
        backtracking by clearing fields with `unset(key)`.
        """
        for key, entry in self.fields.items():
            if not entry["filled"]:
                return key
        return None

    def summary(self):
        """Return a nice printable summary of all filled fields."""
        lines = []
        for k, v in self.fields.items():
            val = v["value"]
            status = "Filled" if v["filled"] else "Empty"
            lines.append(f"{status} {k}: {val}")
        return "\n".join(lines)

#### Functions

In [28]:
def read_input(prompt: str) -> str:
    """
    Centralized input handler with built-in 'exit' and 'back' controls.

    Parameters:
    ----------
    prompt : str
        The prompt to display to the user.
    Returns:
    -------
    str
        The user input, or special command indicators.
    """
    # strip leading/trailing whitespace and convert to lower case for command checks
    raw = input(prompt).strip()
    lower = raw.lower()

    # check for special commands
    if lower in ("exit","quit"): return "__EXIT__"
    if lower == "back": return "__BACK__"

    # return input cleaned of leading/trailing whitespace and lowercased
    return lower

##### Backend - Execution

In [29]:
def prepare_download(save_dir: str,
                     filename_base: str,
                     year: int,
                     month: int,
                     skip_all: bool = False,
                     overwrite_all: bool = False,
                     case_by_case: bool = False,
                     skipped_downloads: list = None,
                     data_file_format: str = "grib"
    ) -> tuple[bool, str]:
    """
    Check if a monthly ERA5 file already exists and decide whether to download.

    Parameters
    ----------
    save_dir : str
        Directory to save the downloaded file.
    filename_base : str
        Base name for the file.
    year : int
        Year of the data to download.
    month : int
        Month of the data to download.
    skip_all : bool, optional
        If True, skip all existing files without prompt, by default False.
    overwrite_all : bool, optional
        If True, overwrite all existing files without prompt, by default False.
    case_by_case : bool, optional
        If True, prompt user for each existing file, by default False.
    skipped_downloads : list, optional
        List to append skipped downloads, by default [].
    data_file_format : str, optional
        File format extension, by default "grib".

    Returns
    -------
    tuple: (download: bool, save_path: str)
        download: Whether to perform the download.
        save_path: Full path for the target file.
    """
    if skipped_downloads is None:
        skipped_downloads = []

    save_path = os.path.join(save_dir, f"{filename_base}_{year}-{month}.{data_file_format}")
    download = True

    # Handle existing file logic
    if os.path.exists(save_path):
        if skip_all:
            tqdm.write(f"Skipping existing file for {year}-{month}: {save_path}")
            skipped_downloads.append((year, month))
            download = False
            return download, save_path

        elif overwrite_all:
            tqdm.write(f"Overwriting existing file for {year}-{month}: {save_path}")
            return download, save_path

        elif case_by_case:
            while True:
                user_input = read_input(
                    f"\nFile already exists for {year}-{month}: {save_path}\n"
                    "Do you want to overwrite it? (y/n): ")

                if user_input in NORMALIZATION_MAP["confirmation"]:
                    break
                print("Invalid input. Please enter 'y' or 'n'.")

            if user_input == "n":
                tqdm.write(f"Skipping existing file for {year}-{month}: {save_path}")
                skipped_downloads.append((year, month))
                download = False
                return download, save_path
            else:
                tqdm.write(f"Overwriting existing file for {year}-{month}: {save_path}")
                return download, save_path

    return download, save_path


In [30]:
def execute_cds_download(
                     session: SessionState,
                     save_path: str,
                     dataset_product_name: str,
                     product_type: str,
                     variables: list[str],
                     days: list[str],
                     times: list[str],
                     grid_area: list[float],
                     year: int,
                     month: int,
                     data_download_format: str = "grib",
                     successful_downloads: list = None,
                     failed_downloads: list = None,
                     max_retries: int = 6,
                     retry_delay_sec: int = 15
    ) -> tuple[int, int, str]:
    """
    Execute a single ERA5 monthly download with retry logic.

    Parameters
    ----------
    session : SessionState
        Session state containing the authenticated CDS API client.
    dataset_product_name : str
        Dataset name (e.g., 'reanalysis-era5-single-levels').
    product_type : str
        Product type (e.g., 'reanalysis').
    variables : list[str]
        List of variable names.
    days : list[str]
        List of days to download.
    times : list[str]
        List of times to download.
    grid_area : list[float]
        Geographic boundaries [north, west, south, east].
    data_download_format : str
        Format of the downloaded data (e.g., 'grib', 'netcdf').
    year : int
        Year of the data to download.
    month : int
        Month of the data to download.
    save_path : str
        Full path to save the downloaded file.
    successful_downloads : list, optional
        List to append successful downloads.
    failed_downloads : list, optional
        List to append failed downloads.
    max_retries : int, optional
        Maximum number of retry attempts, by default 6.
    retry_delay_sec : int, optional
        Delay in seconds between retries, by default 15.

    Returns
    -------
    (year, month, status): tuple
        status = "success" | "failed"
    """
    if successful_downloads is None:
        successful_downloads = []
    if failed_downloads is None:
        failed_downloads = []

    cds_client_session = session.get("session_client")

    month_start = time.time()
    for attempt in range(1, max_retries + 1):
        try:
            tqdm.write(f"\tAttempt {attempt} of {max_retries} for {year}-{month}...")
            cds_client_session.retrieve(
                dataset_product_name,
                {
                    "product_type": [product_type],
                    "variable": variables,
                    "year": str(year),
                    "month": [month],
                    "day": days,
                    "time": times,
                    "area": grid_area,
                    "format": data_download_format,
                },
                save_path,
            )
            elapsed = time.time() - month_start
            tqdm.write(f"SUCCESS: {year}-{month} in {format_duration(elapsed)}")

            successful_downloads.append((year, month))
            return (year, month, "success")

        except Exception as e:
            tqdm.write(f"WARNING: Attempt {attempt} failed for {year}-{month}: {e}")
            if attempt < max_retries:
                tqdm.write(f"\tWaiting {retry_delay_sec} seconds before retrying...")
                time.sleep(retry_delay_sec)
                try:
                    api_url, api_key = session.get("api_url"), session.get("api_key")
                    creds_dict = {"url": api_url, "key": api_key}
                    cds_client_session_new = ensure_cds_connection(cds_client_session, creds_dict)

                    tqdm.write(f"\tRe-authenticated CDS API client.")
                    session.set("session_client", cds_client_session_new)

                except Exception as auth_e:
                    tqdm.write(f"\tWARNING: Re-authentication failed: {auth_e}")
            else:
                tqdm.write(f"FAILURE: all {max_retries} attempts failed for {year}-{month}.")
                failed_downloads.append((year, month))
                return (year, month, "failed")


In [31]:
def download_cds_month(session: SessionState,
                   dataset_product_name: str,
                   product_type: str,
                   variables: list[str],
                   days: list[str],
                   times: list[str],
                   grid_area: list[float],
                   data_download_format: str,
                   save_dir: str,
                   filename_base: str,
                   year: int,
                   month: int,
                   skip_all: bool = False,
                   overwrite_all: bool = False,
                   case_by_case: bool = False,
                   skipped_downloads: list = None,
                   successful_downloads: list = None,
                   failed_downloads: list = None,
                   max_retries: int = 6,
                   retry_delay_sec: int = 10
    ) -> tuple[int, int, str]:
    """
    Orchestrate ERA5 monthly download: handle file checks, then execute download.

    Parameters
    ----------
    Combines parameters from `prepare_download` and `execute_download`.

    Returns
    -------
    (year, month, status): tuple
        status = "success" | "failed" | "skipped"

    """

    proceed, save_path = prepare_download(
        save_dir, filename_base, year, month,
        skip_all, overwrite_all, case_by_case, skipped_downloads
    )
    if not proceed:
        return (year, month, "skipped")

    return execute_cds_download(
        session=session, save_path=save_path, dataset_product_name=dataset_product_name, product_type=product_type, variables=variables, days=days,
        times=times, grid_area=grid_area, year=year, month=month, data_download_format=data_download_format,
        successful_downloads=successful_downloads, failed_downloads=failed_downloads, max_retries=max_retries, retry_delay_sec=retry_delay_sec
    )

##### Utilities: Warnings & Systems Checks

In [32]:
def check_existing_files(
        save_dir: Path,
        filename_base: str,
        start_date: str,
        end_date: str,
    ) -> int:
    """
    Check for existing monthly files for the given configuration between
    start_date and end_date (inclusive months).

    Parameters
    ----------
    save_dir : Path
        Directory to save the downloaded file.
    filename_base : str
        Base name for the file (usually includes configuration hash).
    start_date : str
        Start date in YYYY-MM-DD format.
    end_date : str
        End date in YYYY-MM-DD format.

    Returns
    -------
    bool
        True if one or more matching files already exist, False otherwise.
    """

    print("\nChecking for existing files...\n" + '-'*30)

   # Convert dates to datetime objects
    try:
        start = datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.strptime(end_date, "%Y-%m-%d")
    except ValueError:
        print("Invalid date format provided to check_existing_files(). Expected YYYY-MM-DD.")
        return False

    # Build list of year-month pairs between start and end (inclusive)
    pairs = []
    current = start
    while current <= end:
        pairs.append((current.year, current.month))
        # advance to next month
        if current.month == 12:
            current = datetime(current.year + 1, 1, 1)
        else:
            current = datetime(current.year, current.month + 1, 1)

    existing_files = []

    # Check each year-month combination for existing files
    for year, month in pairs:
        pattern_prefix = f"{filename_base}_{year}_{month:02d}"
        for f in os.listdir(save_dir):
            if f.startswith(pattern_prefix):
                existing_files.append(os.path.join(save_dir, f))

    # Report results
    if existing_files:
        print(f"-> Found {len(existing_files)} existing files in {save_dir} matching configuration.")
        sample_files = existing_files[:2]
        for sf in sample_files:
            print(f"\t - {sf}")
        if len(existing_files) > 2:
            print(f"...and {len(existing_files) - 2} more.")
        return True
    else:
        print(f"-> No existing files found for configuration [{filename_base}] "
              f"between {start_date} and {end_date}.")
        return False

In [33]:
def prompt_skip_overwrite_files(session: SessionState) -> str:
    """
    Prompt user to choose skip/overwrite/case-by-case for existing files.

    Parameters
    ----------
    session : SessionState
        Session state to store user choice.

    Returns
    -------
    str
        One of "overwrite_all", "skip_all", "case_by_case"
    """

    print("\nChoose how to proceed with existing files:\n")
    print("\t1. Overwrite all existing files")
    print("\t2. Skip all existing files")
    print("\t3. Case-by-case confirmation")

    while True:
        print("")
        choice = read_input("Enter choice (1/2/3): ")

        if choice in ("__EXIT__", "__BACK__"):
            return choice

        if choice in ["1", "2", "3"]:
            break
        print("Invalid input. Please enter 1, 2, or 3.")

    if choice == "1":
        session.set("existing_file_action", "overwrite_all")
    elif choice == "2":
        session.set("existing_file_action", "skip_all")

    else:
        session.set("existing_file_action", "case_by_case")

    print(f"You selected option {choice} - {session.get('existing_file_action')}")

    return session.get("existing_file_action")

In [5]:
def estimate_grib_size_mb(
        variables_list: list[str],
        area_bounds_list: list[float],
        size_mb_per_var_global: float = 80,
    ) -> float:
    """
    Roughly estimate file size per month in MB

    Parameters
    ----------
    variables_list : list[str]
        List of variable names.
    area_bounds_list : list[float]
        List of boundaries [north, west, south, east].
    size_mb_per_var_global : float
        Empirical size in MB per variable for global data per month.
        Default is 80 MB.

    Returns
    -------
        float: Estimated file size in MB.
    """
    num_vars = len(variables_list)
    north, west, south, east = area_bounds_list

    lat_span = abs(north - south)
    lon_span = abs(east - west)

    area_fraction = (lat_span * lon_span) / (180 * 360)  # fraction of global

    base_size_per_var_global = size_mb_per_var_global

    return num_vars * base_size_per_var_global * area_fraction


In [ ]:
def estimate_era5_download(
        variables: list[str],
        area: list[float],
        size_mb_per_var_global: float = 80,
        observed_speed_mbps: float = 25.0,
    ) -> tuple[float, float]:
    """
    Estimate GRIB size (MB) and total wall time (min) for CDS API requests.

    Parameters
    ----------
    variables : list[str]
        List of variable names.
    area : list[float]
        List of boundaries [north, west, south, east].
    size_mb_per_var_global : float
        Empirical size in MB per variable for global data per month.
        Default is 80 MB.
    observed_speed_mbps : float
        Observed download speed in megabits per second. Default is 25.0 Mbps.

    Returns
    -------
        tuple: Estimated file size in MB and estimated time in minutes.
    """
    n_vars = len(variables)
    north, west, south, east = area
    area_fraction = (abs(north - south) * abs(east - west)) / (180 * 360)

    # Estimate
    est_size_mb = size_mb_per_var_global * area_fraction * n_vars

    # Convert observed Mbps to MB/s
    download_mb_per_s = observed_speed_mbps / 8.0
    transfer_time_s = est_size_mb / download_mb_per_s

    # Add fixed overhead for CDS preparation and throttling delays
    # 180 s server prep + 0.2 min/var (~12 s/var)
    overhead_s = 180 + (12 * n_vars)

    total_time_s = transfer_time_s + overhead_s
    total_time_min = total_time_s / 60

    return est_size_mb, total_time_min

In [ ]:
def internet_speedtest(
        test_urls: list[str],
        max_seconds: int = 15,
        ) -> float:
    """
    Download ~100MB test file from a fast CDN to estimate speed (MB/s).

    Parameters
    ----------
    test_urls : list[str], optional
        List of URLs of the test files.
    max_seconds : int, optional
        Maximum time to wait for a response, by default 15 seconds.

    Returns
    -------
        float: Estimated download speed in Mbps.
    """
    if test_urls is None:
        test_urls = [
            "https://speedtest.london.linode.com/100MB-london.bin",
            "https://mirror.de.leaseweb.net/speedtest/100mb.bin",
            "https://ipv4.download.thinkbroadband.com/100MB.zip",
        ]
    print("="*40)
    print(f"Internet speed test\n" + "="*40)
    print("\n")
    for url in test_urls:
        try:
            print(f"Testing {url}\n\tMay take up to ~{max_seconds}s...")
            t0 = time.time()
            downloaded = 0
            with requests.get(url, stream=True, timeout=max_seconds) as r:
                r.raise_for_status()
                for chunk in r.iter_content(chunk_size=1024 * 1024):  # 1 MB chunks
                    if not chunk:
                        break
                    downloaded += len(chunk)
                    if (time.time() - t0) > max_seconds or downloaded >= 50_000_000:
                        break
            elapsed = max(1e-6, time.time() - t0)
            mbps = (downloaded * 8 / 1e6) / elapsed
            print(f"\tRESULT: SUCCESS — {mbps:.1f} Mbps (based on {downloaded/1e6:.1f} MB in {elapsed:.1f}s)")
            return float(mbps)
        except Exception as e:
            print(f"\tRESULT: FAILURE on {url}: {e}")
    print("All tests failed. Assuming 25 Mbps.")
    return 25.0


In [ ]:
def generate_filename_hash(
        dataset: str,
        variables : list[str],
        boundaries : list[float]
    ) -> str:
    """
    Generate a unique hash for the download parameters that will be used to create the filename.

    Parameters
    ----------
    dataset_short_name : str
        The dataset short name (era5-world etc).
    variables : list[str]
        List of variable names.
    boundaries : list[float]
        List of boundaries [north, west, south, east].

    Returns
    -------
        str: A unique hash string representing the download parameters.
    """
    # Create unique string from all parameters
    param_string = f"{dataset}|{sorted(variables)}|{boundaries}"

    # Generate hash
    hash_object = hashlib.md5(param_string.encode())
    return hash_object.hexdigest()[:12]  # 12 characters


In [ ]:
def ensure_cds_connection(client: cdsapi.Client,
                          creds: dict,
                          max_reauth_attempts: int = 6,
                          wait_between_attempts: int = 15) -> cdsapi.Client | None:
    """
    Ensure a valid CDS API client. Re-authenticate automatically if the connection drops.

    Parameters
    ----------
    client : cdsapi.Client
        Current CDS API client.
    creds : dict
        {'url': str, 'key': str} stored from initial login.
    max_reauth_attempts : int
        Maximum reconnection attempts before aborting.
    wait_between_attempts : int
        Wait time (seconds) between re-auth attempts.

    Returns
    -------
    cdsapi.Client | None
        Valid client or None if re-authentication ultimately fails.
    """
    for attempt in range(1, max_reauth_attempts + 1):
        try:
            # test if still alive (by touching internal session)
            _ = client.session
            return client
        except Exception as e:
            print(f"Lost connection to CDS API ({e}). Attempting re-authentication {attempt}/{max_reauth_attempts}...")
            try:
                new_client = cdsapi.Client(url=creds["url"], key=creds["key"], quiet=True)
                print("\tRe-authentication successful!")
                return new_client
            except Exception as reauth_e:
                print(f"\tRe-authentication failed! : {reauth_e}")
                if attempt < max_reauth_attempts:
                    print(f"Retrying in {wait_between_attempts} seconds...")
                    time.sleep(wait_between_attempts)
                else:
                    print("\tMaximum reconnection attempts reached. Aborting process.")
                    return None

##### Data Normalisation

In [ ]:
def normalize_input(
        value: str,
        category: str
    ) -> str:
    """
    Normalize user input to canonical internal value as defined in NORMALIZATION_MAP.

    Parameters
    ----------
    value : str
        The user input value to normalize.
    category : str
        The category of normalization (e.g., 'data_provider', 'dataset_short_name')

    Returns
    -------
    str
        The normalized value.

    """
    if not isinstance(value, str):
        return value
    v = value.strip().lower()
    return NORMALIZATION_MAP.get(category, {}).get(v, v)


##### Data Validation

In [ ]:
def validate_data_provider(provider: str) -> bool:
    """
    Ensure dataprovider is recognized and implemented.

    Parameters
    ----------
    provider : str
        Name of the data provider.
    Returns
    -------
        bool: True if valid, False otherwise.
    """
    if provider not in ("cds", "open-meteo"):
        return False
    return True


In [ ]:
def validate_dataset_short_name(dataset_short_name: str, provider: str) -> bool:
    """Check dataset compatibility with provider."""
    if provider == "cds" and dataset_short_name not in (
        "era5-world",
        # "era5-land",      - Not yet implemented
    ):
        return False
    if provider == "open-meteo":
        return False    # Not yet implemented
    return True

In [ ]:
def validate_cds_api_key(url: str, key: str) -> cdsapi.Client | None:
    """
    Validate CDS API credentials by attempting to initialize a cdsapi.Client.

    Parameters
    ----------
    url : str
        CDS API URL.
    key : str
        CDS API key.

    Returns
    -------
    cdsapi.Client | None
        Authenticated client if successful, otherwise None.
    """
    try:
        print("Testing CDS API connection with provided credentials...")
        client = cdsapi.Client(url=url, key=key, quiet=True)
        print("\tAuthentication successful!")
        return client
    except Exception as e:
        print(f"\tAuthentication failed: {e}")
        return None

In [ ]:
def validate_date(value: str) -> bool:
    """
    Validate date format as YYYY-MM-DD.

    Parameters
    ----------
    value : str
        Date string to validate.

    Returns
    -------
    bool
        True if valid, False otherwise.
            """
    try:
        datetime.strptime(value, "%Y-%m-%d")
        return True
    except ValueError:
        return False

In [ ]:
def validate_coordinates(
        north: int | float,
        south: int | float,
        east: int | float,
        west: int | float
    ) -> bool:
    """
    Ensure coordinates are within realistic bounds.

    Parameters
    ----------
    north : int | float
        Northern latitude boundary.
    south : int | float
        Southern latitude boundary.
    east : int | float
        Eastern longitude boundary.
    west : int | float
        Western longitude boundary.

    Returns
    -------
    bool
        True if coordinates are valid, False otherwise.
    """
    return all([
        -90 <= south <= 90,
        -90 <= north <= 90,
        -180 <= west <= 180,
        -180 <= east <= 180,
        north > south
    ])

In [ ]:
def validate_variables(
        variable_list: list[str],
        variable_restrictions: list[str],
        restriction_allow: bool = False
    ) -> bool:
    """
    Ensure user-specified variables are available for this dataset.

    Parameters
    ----------
    variable_list : list[str]
        List of variable names to validate.
    variable_restrictions : list[str]
        List of variables that are either allowed or disallowed.
    restriction_allow : bool
        If True, variable_restrictions is an allowlist (i.e. in). If False, it's a denylist
        (i.e. not in)

    Returns
    -------
    bool
        True if all variables are valid, False otherwise.
    """
    return all(v in variable_restrictions for v in variable_list) if restriction_allow else all(v not in variable_restrictions for v in variable_list)


In [ ]:
def validate_directory(path: str) -> bool:
    """
    Check if path exists or can be created.

    Parameters
    ----------
    path : str
        Directory path to validate.

    Returns
    -------
    bool
        True if path exists or was created successfully, False otherwise.
    """
    p = Path(path)
    if p.exists():
        return True
    try:
        p.mkdir(parents=True, exist_ok=True)
        return True
    except Exception:
        return False


In [ ]:
def validate_numeric(
        value: str | int | float | bool | None,
        min_value=None,
        max_value=None,
        value_type=int
    ) -> bool:
    """
    Generic numeric validator.

    Parameters
    ----------
    value : str | int | float | bool | None
        Value to validate.
    min_value : int | float, optional
        Minimum acceptable value.
    max_value : int | float, optional
        Maximum acceptable value.
    value_type : type, optional
        Type to cast value to (int or float), by default int.
    Returns
    -------
    bool
        True if value is valid, False otherwise.
    """
    if value is None or value == "":
        return False

    try:
        num = value_type(value)

        # range checks
        if (min_value is not None and num < min_value):
            return False
        if (max_value is not None and num > max_value):
            return False

        return True
    except (ValueError, TypeError):
        return False


In [ ]:
def clamp_era5_available_end_date(end: datetime) -> datetime:
    """
    Clamp end date to ERA5 data availability boundary (8 days ago).

    Parameters
    ----------
    end : datetime
        Desired end date.

    Returns
    -------
        datetime: Clamped end date.

    NOTES: ERA5 data is available up to 8 days prior to the current date.
    8-day lag is used to ensure data availability.

    """
    EIGHT_DAY_LAG = 8
    upper = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=EIGHT_DAY_LAG)
    if end > upper:
        print(f"Adjusting end date from {end.date()} to data availability boundary {upper.date()} (−{EIGHT_DAY_LAG} days).")
        return upper
    return end

In [9]:
def format_duration(seconds: float) -> str:
    """
    Convert seconds to a nice Hh Mm Ss string (with decimal seconds).

    Parameters
    ----------
    seconds : float
        Duration in seconds.

    Returns
    -------
        str: Formatted duration string.
    """
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = seconds % 60  # keep remainder as float
    if hours > 0:
        return f"{hours}h {minutes}m {secs:.1f}s"
    elif minutes > 0:
        return f"{minutes}m {secs:.1f}s"
    else:
        return f"{secs:.1f}s"

In [ ]:
def month_days(
        year: int,
        month: int
) -> List[str]:
    last = calendar.monthrange(year, month)[1]
    return [f"{d:02d}" for d in range(1, last + 1)]

In [ ]:
def format_coordinates_nwse(bounds: List[float]) -> str:

    """
    Extracts and formats coordinates as integers in N-W-S-E order

    Parameters
    ----------
    boundaries : list
        List of boundaries in the order [north, west, south, east]
    Returns
    -------
        str: Formatted string in the format 'N{north}W{west}S{south}E{east}'
    """
    n, w, s, e = bounds
    # compact string for filenames
    return f"N{int(n)}W{int(w)}S{int(s)}E{int(e)}"

##### Prompts

In [ ]:
def prompt_data_provider(session: SessionState) -> str:
    """
    Prompt user for which data provider to use (CDS or Open-Meteo).

    Parameters
    ----------
    session : SessionState
        Current session state to store selected data provider.

    Returns
    -------
    str
        Normalized provider name ("cds" or "open-meteo"),
        or special control token "__BACK__" or "__EXIT__".

    """
    print("\nAvailable Data providers:")
    print("\t1. Copernicus Climate Data Store (CDS)")
    print("\t2. Open-Meteo")
    while True:
        raw = read_input("Please enter the data provider you would like to use (name or number): ").strip()
        if raw in ("__EXIT__", "__BACK__"):
            return raw

        data_provider = normalize_input(raw, "data_provider")

        if not validate_data_provider(data_provider):
            print("Invalid provider. Please enter '1' for CDS or '2' for Open-Meteo")
            continue
        if data_provider == "open-meteo":
            print("Open-Meteo support is not yet implemented. Please select CDS.")
            continue

        print(f"You selected: {data_provider.upper()}")
        session.set("data_provider", data_provider)

        return data_provider

In [ ]:
def prompt_dataset_short_name(session: SessionState, provider: str) -> str:
    """
    Prompt for dataset choice.

    Parameters
    ----------
    session: SessionState
        Current session state to store selected dataset.
    provider : str
        Data provider name.

    Returns
    -------
        str: Normalized dataset name or 'exit' / 'back'.

    """
    if provider != "cds":
        print("Currently only CDS datasets are supported.")
        return "__BACK__"

    print("\nAvailable datasets:\n" + '-'*30)
    if provider == "cds":
        print("\t1. ERA5-Land")
        print("\t2. ERA5-World")

    if provider == "open-meteo":
        print("\t(Options not yet implemented)")

    while True:
        raw = read_input("Please enter the dataset you would like to use (name or number): ").strip()
        if raw in ("__EXIT__", "__BACK__"):
            return raw

        if provider == "cds":
            dataset_short_name = normalize_input(raw, "era5_dataset_short_name")

        # Not needed currently, but for future expansion
        elif provider == "open-meteo":
            dataset_short_name = normalize_input(raw, "open_meteo_dataset_short_name")

        if not validate_dataset_short_name(dataset_short_name, provider):
            print("Invalid or unsupported dataset. Try again.")
            continue
        if dataset_short_name != "era5-world":
            print("Only ERA5-World dataset is implemented in this version. Please select ERA5-World.")
            continue

        print(f"You selected: {dataset_short_name.upper()}")
        session.set("dataset_short_name", dataset_short_name)

        return dataset_short_name

In [ ]:
def prompt_cds_url(session: SessionState, api_url_default: str = "https://cds.climate.copernicus.eu/api") -> str:
    """
    Prompt for CDS API URL.

    Parameters
    ----------
    session : SessionState
        Current session state to store API URL.
    api_url_default : str
        Default CDS API URL. https://cds.climate.copernicus.eu/api

    Returns
    -------
        str: CDS API URL or 'exit' / 'back'.

    """
    print("\nCDS API URL:\n" + '-'*30)
    print(f"Default: {api_url_default}")
    while True:
        raw = read_input("Enter CDS API URL (press Enter to keep default): ").strip()
        if raw in ("__EXIT__", "__BACK__"):
            return raw
        url = raw or api_url_default
        session.set("api_url", url)
        print(f"CDS API URL set to: {url}")
        return url

In [ ]:
def prompt_cds_api_key(session: SessionState) -> str:
    """
    Prompt only for the CDS API key (hidden input).

    Parameters
    ----------
    session : SessionState
        Current session state to store API key.
    Returns
    -------
    str
        CDS API key or 'exit' / 'back'.
    """
    print("\nCDS API Key:")
    print("(Type 'back' to go to the previous step or 'exit' to quit.)")
    while True:
        key = getpass.getpass("Enter your CDS API key: ").strip()
        if key.lower() in ("exit", "quit"):
            return "__EXIT__"
        if key.lower() == "back":
            return "__BACK__"
        if not key:
            print("No API key entered. Please try again.")
            continue
        session.set("api_key", key)
        print(f"You entered an API key of length {len(key)} characters.")
        return key

In [ ]:
def prompt_cds_authentication(
        api_url_default: str = "https://cds.climate.copernicus.eu/api",
        connection_attempts: int = 5
    ) -> dict | None:
    """
    Prompt the user for CDS API credentials and attempt connection.

    Parameters
    ----------
    api_url_default : str
        Default CDS API URL.
    connection_attempts : int
        Maximum number of connection attempts.
        Default is 5.

    Returns
    -------
    dict | None
        Dictionary with 'url', 'key', and 'client' if successful,
    """
    print("=" * 40 + "\nCDS API Authentication\n" + "=" * 40)

    max_attempts = connection_attempts
    for attempt in range(1, max_attempts + 1):

        # UPDATED: split into URL and Key prompts
        url = read_input(f"[Attempt {attempt}/{max_attempts}] Enter API URL (press Enter to keep default): ").strip() or api_url_default
        if url == "__EXIT__":
            return "__EXIT__"
        if url == "__BACK__":
            return "__BACK__"

        key = getpass.getpass("Enter your CDS API key (hidden input, or type 'exit' to quit): ").strip()
        if key.lower() in ("exit", "quit"):
            return "__EXIT__"
        if key.lower() == "back":
            return "__BACK__"
        if not key:
            continue

        client = validate_cds_api_key(url, key)
        if client is not None:
            return {"url": url, "key": key, "client": client}

        if attempt < max_attempts:
            print("Please check your credentials and try again.\n")
        else:
            print("Maximum authentication attempts reached. Exiting.")
            sys.exit(1)

    return None

In [ ]:
def prompt_config_source(session: SessionState) -> str:
    """
    Ask user if they want to load config from file or enter manually.

    Parameters
    ----------
    session : SessionState
        Current session state.

    """
    print("\nWould you like to:")
    print("  1. Load configuration from file (download_request.json)")
    print("  2. Enter configuration manually")
    while True:
        raw = read_input("Enter 1 or 2 (or 'exit'): ").strip().lower()
        if raw in ("__EXIT__", "__BACK__"):
            return raw


        if raw not in ("1", "2"):
            print("Invalid choice. Please enter 1 or 2.")
            continue

        if raw == "1" or raw == "2":
            break

    print(f"You selected option {raw}.")
    session.set("config_source", raw)

    return None

In [ ]:
def prompt_date_range(session: SessionState) -> tuple[str, str]:
    """
    Ask user for start and end date, with validation.

    Parameters
    ----------
    session : SessionState
        Current session state to store date range.

    Returns
    -------
        tuple: (start_date_str, end_date_str) or ("__EXIT__", "__EXIT__") / ("__BACK__", "__BACK__")


    """
    print("\nDate Range Selection:\n" + '-'*30)
    while True:
        start_raw = read_input("Enter start date (YYYY-MM-DD): ")
        if start_raw in ("__EXIT__", "__BACK__"):
            return start_raw, start_raw
        end_raw = read_input("Enter end date (YYYY-MM-DD): ")
        if end_raw in ("__EXIT__", "__BACK__"):
            return end_raw, end_raw

        if not (validate_date(start_raw) and validate_date(end_raw)):
            print("Invalid date format. Use YYYY-MM-DD.")
            continue

        start = datetime.strptime(start_raw, "%Y-%m-%d")
        end = datetime.strptime(end_raw, "%Y-%m-%d")
        if end <= start:
            print("ERROR: End date must be after start date.")
            continue

        # Clamp end date for CDS
        if session.get("data_provider") == "cds":
            end = clamp_era5_available_end_date(end)

        print(f"Selected date range: {start.date().isoformat()} to {end.date().isoformat()}")
        session.set("start_date", start.date().isoformat())
        session.set("end_date", end.date().isoformat())

        return start.date().isoformat(), end.date().isoformat()

In [ ]:
def prompt_coordinates(session: SessionState) -> list[float]:
    """
    Prompt user for geographic boundaries (N, S, W, E) with validation.

    Parameters
    ----------
    session : SessionState
        Current session state to store geographic boundaries.

    Returns
    -------
    list
        [north, west, south, east] boundaries or special tokens "__EXIT__" / "__BACK__".
    """
    print("\nGrid Area Selection (ESPG: 4326):\n" + '-'*30)
     while True:
        entries = {}
        for label, key in [("Northern latitude", "north"),
                           ("Southern latitude", "south"),
                           ("Western longitude", "west"),
                           ("Eastern longitude", "east")]:
            value = read_input(f"Enter {label} boundary: ")
            if value in ("__BACK__", "__EXIT__"):
                return value
            entries[key] = value

        try:
            n, s, w, e = (
                float(entries["north"]),
                float(entries["south"]),
                float(entries["west"]),
                float(entries["east"]),
            )
        except ValueError:
            print("Please enter numeric values for all coordinates.")
            continue

        if not validate_coordinates(n, s, e, w):
            print("Invalid bounds. Check that -90 ≤ lat ≤ 90, -180 ≤ lon ≤ 180, and North > South.")
            continue

        bounds = [n, w, s, e]
        print(f"You selected: N{n}, W{w}, S{s}, E{e}")
        session.set("region_bounds", bounds)
        return bounds

In [ ]:
def prompt_variables(
        session: SessionState,
        variable_restrictions_list: list[str],
        restriction_allow: bool = False
    ) -> list[str] | str:
    """
    Ask for variables to download, validate each against allowed/disallowed list,
    and only update session if the full set is valid.

    Parameters
    ----------
    session : SessionState
        Current session state to store selected variables.
    variable_restrictions_list : list[str]
        List of variables that are either allowed or disallowed.
    restriction_allow : bool
        If True, variable_restrictions_list is an allowlist (i.e. in).
        If False, it's a denylist (i.e. not in)

    Returns
    -------
    list[str] | str
        List of selected variable names, or control token "__BACK__" / "__EXIT__".
    """

    print(f"\nVariable Selection [{session.get('dataset_short_name')}]:\n" + "-" * 30)
    print("(Type 'back' to return to previous step or 'exit' to quit.)")

    while True:
        raw = read_input("Enter variable names (comma-separated): ").strip()

        # ---- Handle navigation first ----
        if raw in ("__EXIT__", "__BACK__"):
            return raw

        # ---- Parse list ----
        variable_list = [v.strip().lower() for v in raw.split(",") if v.strip()]

        if not variable_list:
            print("Please enter at least one variable name.")
            continue

        # ---- Validate list using new function ----
        all_valid = validate_variables(variable_list, variable_restrictions_list, restriction_allow)

        if not all_valid:
            if restriction_allow:
                # allowlist mode: only these variables are valid
                valid_vars = [v for v in variable_list if v in variable_restrictions_list]
                invalid_vars = [v for v in variable_list if v not in variable_restrictions_list]
                print("Some variables are not recognized or not available for this dataset:")
                for iv in invalid_vars:
                    print(f"   - {iv}")
            else:
                # denylist mode: disallowed variables
                valid_vars = [v for v in variable_list if v not in variable_restrictions_list]
                invalid_vars = [v for v in variable_list if v in variable_restrictions_list]
                print("The following variables are known to cause issues or are disallowed:")
                for iv in invalid_vars:
                    print(f"   - {iv}")

            # ---- Offer to continue with valid ones ----
            if valid_vars:
                proceed = read_input(
                    f"\nWould you like to proceed with only the valid variables ({', '.join(valid_vars)})? (y/n): "
                ).strip().lower()
                if proceed.startswith("y"):
                    print("Proceeding with valid subset.\n")
                    session.set("variables", valid_vars)
                    return valid_vars
                else:
                    print("Let's try again.\n")
                    continue
            else:
                print("No valid variables remain. Please try again.\n")
                continue

        # ---- If everything is valid ----
        print(f"You selected {len(variable_list)} valid variables:")
        print(", ".join(variable_list))
        confirm = read_input("Confirm selection? (y/n): ").strip().lower()
        if confirm.startswith("y"):
            session.set("variables", variable_list)
            return variable_list
        else:
            print("Let's try again.\n")

In [ ]:
def prompt_save_directory(default_dir: Path) -> Path | str:
    """
    Ask for save directory, create if necessary.

    Parameters
    ----------
    default_dir : Path
        Default directory to suggest.

    Returns
    -------
    Path | str
        Path to save directory, or control token "__BACK__" / "__EXIT__".

    """
    print("\nSave Directory:")
    print(f"Default: {default_dir}")
    print("(Type 'back' to go to the previous step or 'exit' to quit.)")
    while True:
        raw = read_input("Enter a path (or press Enter to use default): ").strip()
        if raw in ("__EXIT__", "__BACK__"):
            return raw
        path = Path(raw or default_dir).expanduser().resolve()
        if validate_directory(str(path)):
            print(f"Using directory: {path}")
            return path
        print("Directory could not be created or accessed. Try another path.")


In [ ]:
def prompt_retry_settings() -> dict | str:
    """
    Ask user for retry limits.

    Parameters
    ----------
    Returns
    -------
    dict | str
        Dictionary with 'max_retries' and 'retry_delay_sec',"""
    print("\nRetry Settings:")
    print("(Type 'back' to go to the previous step or 'exit' to quit.)")
    while True:
        raw = read_input("Use defaults? Retries=6, Delay=15s (y/n): ").strip().lower()
        if raw in ("__EXIT__", "__BACK__"): return raw
        if raw.startswith("y"):
            return {"max_retries": 6, "retry_delay_sec": 15}
        try:
            mr = read_input("Max retries (int): ").strip()
            if mr in ("__EXIT__", "__BACK__"): return mr
            mr = int(mr or 6)
            rd = read_input("Retry delay seconds (int): ").strip()
            if rd in ("__EXIT__", "__BACK__"): return rd
            rd = int(rd or 15)
            return {"max_retries": mr, "retry_delay_sec": rd}
        except ValueError:
            print("Please enter integer values.")

In [ ]:

def prompt_continue_confirmation(summary_text: str) -> bool:
    """Display summary and confirm before starting downloads."""
    ...

In [ ]:
def load_data_download_request(
        file_path: str = None
        ) -> dict:
    """
    Load configuration from JSON requirements file.

    Parameters
    ----------
    file_path : str
        Path to the JSON requirements file.
    Returns
    -------
    dict
        Configuration dictionary loaded from the JSON file.
    """
    if file_path is None:
        file_path = os.path.join(root_dir, "input", "download_request.json")

    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Requirements file not found: {file_path}")

    try:
        with open(file_path, "r") as f:
            config = json.load(f)

        print(f"SUCCESS: Loaded configuration from {file_path}")
        return config

    except json.JSONDecodeError as e:
        print(f"FAILURE: Could not load requirements from {file_path}")
        raise ValueError(f"Error message: {e}")

In [12]:
def interactive_config(
        base_config: dict,
        input_labels: dict,
        data_retrieval_instructions_filepath: str
    ) -> dict:
    """
    Loop through dictionary of configuration labels and prompt user for input.
    The user can enter 'req' to revert to the requirements.json value.
    """
    print("=" * 60)
    print("🧭 ERA5 Data Download Configuration")
    print("=" * 60)
    print("\n💡 Enter values or press Enter to keep defaults from file.")
    print(f"   Type 'req' to revert to values from {data_retrieval_instructions_filepath} for that field.\n")

    config = base_config.copy()

    for key, label in input_labels.items():
        # Handle special cases:
        if key not in base_config:
            print(f"Skipping unknown key: {key}")
            continue

        # Skip or customize prompt style for certain keys
        if key == "api_key":
            config[key] = prompt_or_default(label, base_config[key], is_secret=True)
        elif key == "variables":
            print(f"\n{label}:")
            print(f"Current variables: {base_config[key]}")
            vars_input = input("> ").strip()
            if vars_input == "" or vars_input.lower() == "req":
                config[key] = base_config[key]
            else:
                config[key] = [v.strip() for v in vars_input.split(",") if v.strip()]
        else:
            config[key] = prompt_or_default(label, base_config[key])

    print("\nConfiguration complete.\n")
    return config


##### Session Management

##### Main

##### Implementation

In [ ]:
def main():
    """
    Orchestrates full data retrieval process.
    Handles:
      - Provider selection
      - Authentication
      - Config source (file/manual)
      - Dataset details (time, space, variables)
      - Estimates + confirmation
      - Parallelisation setup
      - Final download execution
    """
    print("=" * 60)
    print("Welcome to the Weather Data Retrieval Tool")
    print("=" * 60)
    print("\nThis tool will guide you through downloading weather data from\n - The Copernicus Climate Data Store (CDS) using the CDS API\n - The Open-Meteo API (not yet implemented)")
    print("\nYou will be prompted to provide information such as:\n - API credentials and connection details, which dataset to access,\n   desired variables, time range, and geographic area.")
    print("\nThe tool will assist you in estimating download sizes and times based on your selections,\n handling parallel downloads, and managing existing files.\n")
    print("\nFor more details on the CDS or Open-Meteo datasets and APIs, please visit their websites:\n - https://cds.climate.copernicus.eu/ \n - https://open-meteo.com/")
    print("\nYou may type 'exit' at any time to quit.\n" + "-"*60 + "\n")

    # Outer control loop to allow re-entry
    session = SessionState()

    # === Wizard-style driver using first_unfilled_key and back/exit handling ===  # NEW
    while True:
        key = session.first_unfilled_key()
        if key is None:
            break  # all done

        # Step dispatch by key
        if key == "data_provider":
            res = prompt_data_provider(session)
            if res == "__EXIT__": return
            if res == "__BACK__":
                # nothing before this; just continue
                continue

        elif key == "dataset_short_name":
            provider = session.get("data_provider")
            res = prompt_dataset_short_name(session, provider)
            if res == "__EXIT__": return
            if res == "__BACK__":
                session.unset("data_provider")
                continue

        elif key == "api_url":
            # Split URL and key into two steps
            res_url = prompt_cds_url(session, "https://cds.climate.copernicus.eu/api")
            if res_url == "__EXIT__": return
            if res_url == "__BACK__":
                session.unset("dataset_short_name")
                continue

        elif key == "api_key":
            res_key = prompt_cds_api_key(session)
            if res_key == "__EXIT__": return
            if res_key == "__BACK__":
                session.unset("api_url")
                continue

            # Validate creds right after collecting them
            client = validate_cds_api_key(session.get("api_url"), session.get("api_key"))
            if client is None:
                print("Authentication failed. Please re-enter your API details.")
                session.unset("api_key")
                # allow user to go back and fix URL too:
                session.unset("api_url")
                continue
            session.set("client", client)  # NEW: store client in session (not listed in fields, ok)

        elif key == "config_source":
            res = prompt_config_source()
            if res == "__EXIT__": return
            if res == "__BACK__":
                session.unset("api_key")
                continue
            session.set("config_source", res)

            if res == "file":
                try:
                    cfg = load_data_download_request()
                    # ⚠️ PLACEHOLDER: apply cfg to session (you may map keys here)
                    # e.g., session.set("start_date", cfg["start_date"]) ...
                    print("Loaded config file. (You can wire values into session here.)")
                except Exception as e:
                    print(f"Failed to load config file: {e}. Switching to manual entry.")
                    session.set("config_source", "manual")

        elif key == "start_date":
            s, e = prompt_date_range(session)
            if s == "__EXIT__": return
            if s == "__BACK__":
                session.unset("config_source")
                continue
            # prompt_date_range already sets both start_date and end_date

        elif key == "region_bounds":
            bounds = prompt_coordinates(session)
            if bounds == "__EXIT__": return
            if bounds == "__BACK__":
                session.unset("start_date")
                session.unset("end_date")
                continue
            # session already set inside prompt

        elif key == "variables":
            # ⚠️ PLACEHOLDER: if you have a canonical allowed list, reference it here.
            variables = prompt_variables(session, invalid_variables=invalid_era5_world_variables)
            if variables and variables[0] == "__EXIT__": return
            if variables and variables[0] == "__BACK__":
                session.unset("region_bounds")
                continue

        elif key == "save_dir":
            save_path = prompt_save_directory(Path("./data/raw"))
            if save_path == "__EXIT__": return
            if save_path == "__BACK__":
                session.unset("variables")
                continue
            session.set("save_dir", str(save_path))

        elif key == "retry_settings":
            rs = prompt_retry_settings()
            if rs in ("__EXIT__", "__BACK__"):
                if rs == "__BACK__":
                    session.unset("save_dir")
                    continue
                return
            session.set("retry_settings", rs)

        elif key == "parallel_settings":
            ps = prompt_parallelisation_settings()
            if ps in ("__EXIT__", "__BACK__"):
                if ps == "__BACK__":
                    session.unset("retry_settings")
                    continue
                return
            session.set("parallel_settings", ps)

        # loop continues until all fields filled

    # === Estimation & Confirmation ===
    print("\nRunning speed test (quick heuristic)...")
    speed_mbps = internet_speedtest(test_urls=None, max_seconds=10)  # UPDATED: pass None to use defaults
    est_size_mb, est_time_min = estimate_era5_download(
        session.get("variables"),
        session.get("region_bounds"),
        observed_speed_mbps=speed_mbps
    )
    summary = (
        f"Provider: {session.get('data_provider').upper()}\n"
        f"Dataset: {session.get('dataset_short_name')}\n"
        f"Area: {session.get('region_bounds')}\n"
        f"Dates: {session.get('start_date')} to {session.get('end_date')}\n"
        f"Variables: {session.get('variables')}\n"
        f"Save Directory: {session.get('save_dir')}\n"
        f"Estimated total size: {est_size_mb:.1f} MB\n"
        f"Estimated time: {est_time_min:.1f} minutes\n"
        f"Measured internet speed: {speed_mbps:.1f} Mbps\n"
    )
    cont = prompt_continue_confirmation(summary)
    if cont in ("__EXIT__", "__BACK__"):
        if cont == "__BACK__":
            # Let user adjust from the last parameter section (parallel settings)
            session.unset("parallel_settings")
            # and fall back into the wizard loop
            return main()
        return

    if not cont:
        print("Aborting download. Exiting.\n")
        return

    # === Execution placeholders ===
    print("\nStarting data download process...")
    # ⚠️ PLACEHOLDER: Build monthly loops, ensure ensure_cds_connection() per month, and call download_cds_month(...)
    # You can derive (year, month) from start_date/end_date and iterate accordingly.
    print("Data retrieval complete.\n")

    print("\nSession Summary:")
    print(session.summary())


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\nUser interrupted. Exiting.")

Welcome to the Weather Data Retrieval Tool

This tool will guide you through downloading weather data from
 - The Copernicus Climate Data Store (CDS) using the CDS API
 - The Open-Meteo API (not yet implemented)

You will be prompted to provide information such as:
 - API credentials and connection details, which dataset to access,
   desired variables, time range, and geographic area.

The tool will assist you in estimating download sizes and times based on your selections,
 handling parallel downloads, and managing existing files.


For more details on the CDS or Open-Meteo datasets and APIs, please visit their websites:
 - https://cds.climate.copernicus.eu/ 
 - https://open-meteo.com/

You may type 'exit' at any time to quit.
------------------------------------------------------------



NameError: name 'prompt_data_provider' is not defined

In [ ]:


# === MAIN DOWNLOAD LOOP WITH PROGRESS BAR ===
session = auth_prompt()

overall_start_time = time.time()
failed_downloads = []
successful_downloads = []
skipped_downloads = []

max_workers = 2  # Keep small to avoid CDS throttling
futures = []

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = []
with tqdm(total=n_files, desc="Monthly ERA5 data requests progress", unit="file") as pbar:

    for year, month in years_months:
        save_path = os.path.join(save_dir, f"{filename_base}_{year}-{month}.grib")
        if os.path.exists(save_path):
            if overwrite_all:
                tqdm.write(f"\nOverwriting existing file for {year}-{month}: {save_path}\n")
            elif skip_all:
                tqdm.write(f"\nSkipping existing file for {year}-{month}: {save_path}\n")
                skipped_downloads.append((year, month))
                pbar.update(1)
                continue
            elif case_by_case:
                while True:
                    user_input = input(f"\nFile already exists for {year}-{month}: {save_path}\nDo you want to overwrite it? (y/n): ").strip().lower()
                    if user_input in ['y', 'n']:
                        break
                    print("Invalid input. Please enter 'y' or 'n'.")
                if user_input == 'n':
                    tqdm.write(f"Skipping existing file for {year}-{month}: {save_path}\n")
                    pbar.update(1)
                    continue
                else:
                    skipped_downloads.append((year, month))
                    tqdm.write(f"Overwriting existing file for {year}-{month}: {save_path}\n")

        tqdm.write(f"\nProcessing {year}-{month} ...\n")
        month_start = time.time()

        success = False
        for attempt in range(1, max_retries + 1):
            try:
                tqdm.write(f"\tAttempt {attempt} of {max_retries} for {year}-{month}...")
                session.retrieve(
                    era5_world_dataset,
                    {
                        "product_type": ["reanalysis"],
                        "variable": variables,
                        "year": str(year),
                        "month": [month],
                        "day": days,
                        "time": times,
                        "area": grid_area,
                        "format": data_download_format,
                    },
                    save_path,
                )
                elapsed = time.time() - month_start
                tqdm.write(f"SUCCESS: Completed {year}-{month} in {format_duration(elapsed)}")
                success = True
                successful_downloads.append((year, month))
                break
            except Exception as e:
                tqdm.write(f"\tWARNING: Attempt {attempt} failed for {year}-{month}: {e}")
                if attempt < max_retries:
                    tqdm.write(f"\tWaiting {retry_delay_sec} seconds before retrying...")
                    time.sleep(retry_delay_sec)
                    try:
                        session = cdsapi.Client()
                        tqdm.write(f"\tSUCCESS: Re-authenticated CDS API client.")
                    except Exception as auth_e:
                        tqdm.write(f"\tWARNING: Re-authentication failed: {auth_e}")
                else:
                    tqdm.write(f"FAILURE - All {max_retries} attempts failed for {year}-{month}: {e}")
                    failed_downloads.append((year, month))

        pbar.update(1)

overall_elapsed = time.time() - overall_start_time

Checking for existing files...
No existing files found. Proceeding with downloads.
CDS API Authentication

CDS API Client initialized successfully.




Monthly ERA5 data requests progress:   0%|          | 0/90 [00:00<?, ?file/s]


Processing 2018-01 ...

	Attempt 1 of 6 for 2018-01...


2025-10-22 17:30:54,277 INFO Request ID is b523ebed-4728-4c9c-be17-d3ef729bf9ab
2025-10-22 17:30:54,767 INFO status has been updated to accepted
2025-10-22 17:31:03,615 INFO status has been updated to running
2025-10-22 17:49:37,105 INFO status has been updated to successful


7c04ecbaa52c9c244a743a4640016322.grib:   0%|          | 0.00/645M [00:00<?, ?B/s]

Monthly ERA5 data requests progress:   1%|          | 1/90 [18:55<28:04:47, 1135.82s/file]

SUCCESS: Completed 2018-01 in 18m 55.8s

Processing 2018-02 ...

	Attempt 1 of 6 for 2018-02...


2025-10-22 17:49:50,267 INFO Request ID is fc2ed7c6-f984-4418-8949-643e74999a8c
2025-10-22 17:49:50,392 INFO status has been updated to accepted
2025-10-22 17:50:04,123 INFO status has been updated to running
Monthly ERA5 data requests progress:   1%|          | 1/90 [19:42<29:13:20, 1182.03s/file]


KeyboardInterrupt: 

In [ ]:
#=== SUMMARY ===
print("\n📊 Download Summary")
print(f"   Total months requested : {n_files}")
print(f"   Successful downloads   : {len(successful_downloads)}")
print(f"   Skipped downloads      : {len(skipped_downloads)}")
print(f"   Failed downloads       : {len(failed_downloads)}")
print(f"   Total time elapsed     : {format_duration(overall_elapsed)}")

if failed_downloads:
    print("   Failed months:", ", ".join([f"{y}-{m}" for y, m in failed_downloads]))

print(f"Files Downloaded to {save_dir}:")
for year, month in successful_downloads:
    print(f"   - {filename_base}_{year}-{month}.grib")


📊 Download Summary
   Total months requested : 3
   Successful downloads   : 1
   Skipped downloads      : 0
   Failed downloads       : 0
   Total time elapsed     : 25.7s
Files Downloaded to /Users/Daniel/Desktop/open-source-marginal-emissions/data/raw:
   - era5_world_N38W68S6E98_010b38615413_2025_01.grib


In [ ]:
base_config = load_data_retrieval_instructions()
config = interactive_config(base_config, input_config_labels, "data_retrieval_instructions.json")

#### Variables

##### Define Variables

#### Functions

##### Weather Dataset Specification

In [ ]:
def weather_dataset_specification_providers_choice():
    print("Please choose a weather data provider from one of the following options:")
    print(weather_data_providers_labels.values())
    choice = input("Enter your choice: ")
    failed_attempts = 0
    while choice not in weather_data_provider_aliases:
        print("Invalid choice. Please enter the exact name or number of the weather data provider.")
        failed_attempts += 1
        print(f"Failed attempts: {failed_attempts}/5")
        if failed_attempts >= 5:
            print("Too many failed attempts. Exiting the program.")
            exit()
        choice = input("\nEnter your choice: ")
    return choice

In [ ]:
def era5_weather_dataset_choice():
    print("Please choose an ERA5 weather dataset from one of the following options:")
    print(era5_weather_datasets_labels.values())
    failed_attempts = 0
    choice = input("Enter your choice: ")
    while choice not in era5_weather_dataset_aliases:
        print("Invalid choice. Please enter the exact name or number of the ERA5 weather dataset.")
        failed_attempts += 1
        print(f"Failed attempts: {failed_attempts}/5")
        if failed_attempts >= 5:
            print("Too many failed attempts. Exiting the program.")
            exit()
        choice = input("\nEnter your choice: ")
    return choice

In [ ]:
def weather_data_specification():
    weather_dataset_specification_intro()
    weather_data_provider = None
    weather_dataset = None
    end_loop = False
    while end_loop == False:
        weather_data_provider = weather_dataset_specification_providers_choice()
        if weather_data_provider == 'Open-Meteo':
            print("Open-Meteo data retrieval is not implemented yet. - Please choose another provider.")
            raise NotImplementedError("Open-Meteo data retrieval is not implemented yet.")
            # Implement Open-Meteo data retrieval here
        elif weather_data_provider == 'ERA5':
            weather_dataset = era5_weather_dataset_choice()
            end_loop = True
        else:
            raise ValueError("All previous input validation failed - recheck logic.")
            end_loop = True
    return weather_data_provider, weather_dataset

: 

In [ ]:
weather_data_specification()


Weather Data Retrieval

Welcome to the Weather Data Retrieval Tool!
This tool allows you to retrieve historical weather data from various providers.
Please choose a weather data provider from one of the following options:
dict_values(['1. Open-Meteo', '2. ERA5'])
Invalid choice. Please enter the exact name or number of the weather data provider.
Failed attempts: 1/5
Invalid choice. Please enter the exact name or number of the weather data provider.
Failed attempts: 2/5
Invalid choice. Please enter the exact name or number of the weather data provider.
Failed attempts: 3/5
Invalid choice. Please enter the exact name or number of the weather data provider.
Failed attempts: 4/5
Invalid choice. Please enter the exact name or number of the weather data provider.
Failed attempts: 5/5
Too many failed attempts. Exiting the program.
Invalid choice. Please enter the exact name or number of the weather data provider.
Failed attempts: 6/5
Too many failed attempts. Exiting the program.
Invalid cho

What data source would you like to use?
* Open-Meteo
* CDS API (ERA5 datasets)
    * IF USING CDS API:
    * What dataset would you like to use?
        * ERA5-Land only
        * ERA5-World only
        * ERA5-Land where available, otherwise ERA5-World
        * ERA5-World where available, otherwise ERA5-Land

Options:
open-meteo
era5-land
era5-world
era5-land-then-era5-world
era5-world-then-era5-land

Time Period:
Over what time period would you like to retrieve data?
* Start date (YYYY-MM-DD):
* End date (YYYY-MM-DD):

How would you like to  specify the goegraphical area over which to retrieve data?
* Provide bounding box coordinates
* Select bounding box on map
* Provide Country Name
* Provide City and Country Name 

Over what geographical area would you like to retrieve data?
* North latitude (degrees):
* South latitude (degrees):
* East longitude (degrees):
* West longitude (degrees):

* What temporal resolution would you like the data at?
    * 2-Hourly (aggregates)
    * Hourly (native)
    * Half-hourly (interpolated)

* What geographic resolution would you like the data at?
    * 0.1° x 0.1° (approx. 11km x 11km) (only native for ERA5-Land)
    * 0.25° x 0.25° (approx. 28km x 28km)
    * 0.5° x 0.5° (approx. 55km x 55km)
    * 1.0° x 1.0° (approx. 111km x 111km)

| NAME | VAR NAME | Units
|------|----------|------|
| Surface pressure | surface_pressure | Pa
| Total cloud cover | total_cloud_cover | 0-1
| 10 metre U wind component | 10m_u_component_of_wind | m/s
| 10 metre V wind component | 10m_v_component_of_wind | m/s
| 2 metre temperature | 2m_temperature | K
| Low cloud cover | low_cloud_cover | 0-1
| Medium cloud cover | medium_cloud_cover | 0-1
| High cloud cover | high_cloud_cover | 0-1
| Instantaneous large-scale surface precipitation fraction | instantaneous_large_scale_surface_precipitation_fraction | 0-1
| 100 metre U wind component | 100m_u_component_of_wind | m/s
| 100 metre V wind component | 100m_v_component_of_wind | m/s
| Surface solar radiation downwards | surface_solar_radiation_downwards | J m**-2
| Surface thermal radiation downwards | surface_thermal_radiation_downwards | J m**-2
| Surface net solar radiation | surface_net_solar_radiation | J m**-2
| Top net solar radiation | top_net_solar_radiation | J m**-2
| Top net thermal radiation | top_net_thermal_radiation | J m**-2
| Top net solar radiation, clear sky | top_net_solar_radiation_clear_sky | J m**-2
| Top net thermal radiation, clear sky | top_net_thermal_radiation_clear_sky | J m**-2
| Surface net solar radiation, clear sky | surface_net_solar_radiation_clear_sky | J m**-2
| Surface net thermal radiation, clear sky | surface_net_thermal_radiation_clear_sky | J m**-2
| TOA incident solar radiation | toa_incident_solar_radiation | J m**-2
| Total sky direct solar radiation at surface | total_sky_direct_solar_radiation_at_surface | J m**-2
| Clear-sky direct solar radiation at surface | clear_sky_direct_solar_radiation_at_surface | J m**-2
| Surface solar radiation downward clear-sky  | surface_solar_radiation_downward_clear_sky	 | J m**-2
| Surface thermal radiation downward clear-sky | surface_thermal_radiation_downward_clear_sky | J m**-2
| Large-scale precipitation | large_scale_precipitation | kg m**-2
| Convective precipitation | convective_precipitation | kg m**-2
| Total precipitation | total_precipitation | kg m**-2
| Total column water | total_column_water | kg m**-2
| Fraction of cloud cover | fraction_of_cloud_cover | 0-1